In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
from zipfile import ZipFile
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix,classification_report
#set notebook display options
pd.options.display.max_rows = 500
pd.options.display.max_columns =500

## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [43]:
#mount drive to load datasets
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
#project path in google drive
project_path = "/content/drive/My Drive/AIML/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
labels = pd.read_csv('labels.csv', encoding = "ISO-8859-1")

In [50]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [51]:
print(labels.breed.value_counts())

scottish_deerhound                126
maltese_dog                       117
afghan_hound                      116
entlebucher                       115
bernese_mountain_dog              114
shih-tzu                          112
pomeranian                        111
great_pyrenees                    111
basenji                           110
samoyed                           109
tibetan_terrier                   107
airedale                          107
leonberg                          106
cairn                             106
japanese_spaniel                  105
beagle                            105
blenheim_spaniel                  102
australian_terrier                102
miniature_pinscher                102
irish_wolfhound                   101
lakeland_terrier                   99
saluki                             99
papillon                           96
whippet                            95
norwegian_elkhound                 95
siberian_husky                     95
pug         

### Get one-hot encodings of labels

In [0]:
targets = pd.Series(labels['breed'])
one_hot_labels = pd.get_dummies(targets, sparse=True)
one_hot_labels = np.asarray(one_hot_labels)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
img_rows = 128
img_cols = 128
num_channels = 3

In [54]:
x_train = []
y_train = []

for f, img in tqdm(labels.values): # f for format ,jpg
    train_img = cv2.imread('./train/{}.jpg'.format(f), 1,)
    train_img_resize = cv2.resize(train_img, (img_rows, img_cols)) 
    x_train.append(train_img_resize)
    y_train.append(img)

100%|██████████| 10222/10222 [00:26<00:00, 387.05it/s]


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
x_train_data = np.asarray(x_train).astype('float32')

In [0]:
x_train_data = x_train_data/255

In [0]:
y_train_data = np.asarray(pd.get_dummies(y_train, sparse=True))

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
train_x, test_x, train_y, test_y = train_test_split(x_train_data, y_train_data, test_size=0.2, random_state=123)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
submission = pd.read_csv('sample_submission.csv',encoding = "ISO-8859-1")
test_img = submission['id']

Run the below code to load the test image files in x_test_feature

In [60]:
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 1)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [00:25<00:00, 398.80it/s]


Normalize the test data and convert it into 4 dimensions

In [0]:
x_test_data = np.asarray(x_test_feature).astype('float32')

In [0]:
x_test_data = x_test_data/255

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
tf.keras.backend.clear_session()
model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='valid', activation='relu', input_shape=(img_rows,img_cols,3,)))
model1.add(tf.keras.layers.BatchNormalization())
model1.add(tf.keras.layers.Dropout(0.30))
model1.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='valid', activation='relu'))
model1.add(tf.keras.layers.BatchNormalization())
model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(256, kernel_initializer = 'he_normal', activation='relu'))
model1.add(tf.keras.layers.Dense(120, activation='softmax'))

In [0]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [65]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 64)      4864      
_________________________________________________________________
batch_normalization (BatchNo (None, 124, 124, 64)      256       
_________________________________________________________________
dropout (Dropout)            (None, 124, 124, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 122, 122, 64)      256       
_________________________________________________________________
flatten (Flatten)            (None, 952576)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               2

### Use batch_size = 128 and epochs = 10 and execute the model

In [66]:
model1.fit(train_x, train_y,
          validation_data=(test_x, test_y),
          batch_size=128, epochs=10 )

Epoch 1/10
64/64 [==============================] - 13s 210ms/step - loss: 52.9603 - accuracy: 0.0176 - val_loss: 7.1321 - val_accuracy: 0.0054
Epoch 2/10
64/64 [==============================] - 12s 194ms/step - loss: 4.1764 - accuracy: 0.3221 - val_loss: 4.7873 - val_accuracy: 0.0108
Epoch 3/10
64/64 [==============================] - 12s 194ms/step - loss: 2.6140 - accuracy: 0.4553 - val_loss: 4.8325 - val_accuracy: 0.0112
Epoch 4/10
64/64 [==============================] - 12s 194ms/step - loss: 2.0249 - accuracy: 0.5541 - val_loss: 5.0967 - val_accuracy: 0.0117
Epoch 5/10
64/64 [==============================] - 12s 194ms/step - loss: 1.6641 - accuracy: 0.6250 - val_loss: 5.3699 - val_accuracy: 0.0108
Epoch 6/10
64/64 [==============================] - 12s 194ms/step - loss: 1.4443 - accuracy: 0.6730 - val_loss: 6.6961 - val_accuracy: 0.0117
Epoch 7/10
64/64 [==============================] - 12s 194ms/step - loss: 1.2668 - accuracy: 0.7083 - val_loss: 6.1560 - val_accuracy: 0.014

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=True,
                                                          featurewise_std_normalization=True,
                                                          rotation_range=20,
                                                          width_shift_range=0.20,
                                                          height_shift_range=0.20,
                                                          horizontal_flip=True)

In [0]:
datagen.fit(train_x)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
train_generator = datagen.flow(train_x, train_y, batch_size=128)

In [0]:
val_generator = datagen.flow(test_x, test_y, batch_size=128)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [71]:
#Fit the model using fit_generator with batch_size 128 and run for 10 epochs
model1.fit_generator(train_generator,
                    steps_per_epoch = train_generator.n//128,
                    validation_data=val_generator,
                    validation_steps=val_generator.n//128,
                    epochs=10)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
63/63 [==============================] - 31s 491ms/step - loss: 9.4648 - accuracy: 0.0123 - val_loss: 126.1069 - val_accuracy: 0.0073
Epoch 2/10
63/63 [==============================] - 31s 493ms/step - loss: 4.8320 - accuracy: 0.0114 - val_loss: 4.8143 - val_accuracy: 0.0115
Epoch 3/10
63/63 [==============================] - 31s 492ms/step - loss: 4.7951 - accuracy: 0.0112 - val_loss: 4.7936 - val_accuracy: 0.0115
Epoch 4/10
63/63 [==============================] - 31s 493ms/step - loss: 4.7913 - accuracy: 0.0109 - val_loss: 4.7939 - val_accuracy: 0.0104
Epoch 5/10
63/63 [==============================] - 31s 492ms/step - loss: 4.7873 - accuracy: 0.0124 - val_loss: 4.7865 - val_accuracy: 0.0109
Epoch 6/10
63/63 [==============================] - 31s 493ms/step - loss: 4.7807 - accuracy: 0.0114 - val_loss: 4.7979 - val_accuracy: 0.0109
Epoch 7/10
63/63 [==============================] - 31s 496ms/st

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [73]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
tf.keras.backend.clear_session()
model_vgg = tf.keras.models.Sequential()
model_vgg.add(base_model)
model_vgg.add(tf.keras.layers.BatchNormalization())
model_vgg.add(tf.keras.layers.Dropout(0.30, name='dropout_1'))
model_vgg.add(tf.keras.layers.Flatten())
model_vgg.add(tf.keras.layers.Dense(1024, kernel_initializer = 'he_normal', activation='relu'))
model_vgg.add(tf.keras.layers.BatchNormalization())
model_vgg.add(tf.keras.layers.Dropout(0.30, name='dropout_2'))
model_vgg.add(tf.keras.layers.Dense(256, kernel_initializer = 'he_normal', activation='relu'))
model_vgg.add(tf.keras.layers.Dense(120, activation='softmax'))

In [75]:
model_vgg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
for layer in base_model.layers:
  layer.trainable = False

In [77]:
model_vgg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

In [0]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.005)
model_vgg.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model_vgg_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='dog_breed_classification_best.h5',
                                                          monitor='val_accuracy',
                                                          save_best_only=True,
                                                          mode='max',
                                                          verbose=1)

In [80]:
model_vgg.fit(train_generator,
              steps_per_epoch=train_generator.n//128,
              validation_data=val_generator,
              validation_steps=val_generator.n//128,
              epochs=10,
              callbacks=[model_vgg_checkpoint])

Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 4.2888 - accuracy: 0.1442
Epoch 00001: val_accuracy improved from -inf to 0.16771, saving model to dog_breed_classification_best.h5
63/63 [==============================] - 32s 501ms/step - loss: 4.2888 - accuracy: 0.1442 - val_loss: 3.6831 - val_accuracy: 0.1677
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 3.1153 - accuracy: 0.2569
Epoch 00002: val_accuracy improved from 0.16771 to 0.23229, saving model to dog_breed_classification_best.h5
63/63 [==============================] - 31s 499ms/step - loss: 3.1153 - accuracy: 0.2569 - val_loss: 3.3316 - val_accuracy: 0.2323
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 2.7661 - accuracy: 0.3157
Epoch 00003: val_accuracy improved from 0.23229 to 0.27187, saving model to dog_breed_classification_best.h5
63/63 [==============================] - 31s 498ms/step - loss: 2.7661 - accuracy: 0.3157 - val_loss: 2.9504 - val_accuracy: 0.271

# Model accuracy is VERY LOW !!!

Try to get training and validation accuracy to be more than 90%

In [81]:
#Make all layers in base_model as Non-trainable
for layer in base_model.layers:
  if (layer.name =='block5_conv3'):
    print(layer.name,"is Trainable")
    layer.trainable = True
  else:
    print(layer.name,"is Non-Trainable")
    layer.trainable = False

input_1 is Non-Trainable
block1_conv1 is Non-Trainable
block1_conv2 is Non-Trainable
block1_pool is Non-Trainable
block2_conv1 is Non-Trainable
block2_conv2 is Non-Trainable
block2_pool is Non-Trainable
block3_conv1 is Non-Trainable
block3_conv2 is Non-Trainable
block3_conv3 is Non-Trainable
block3_pool is Non-Trainable
block4_conv1 is Non-Trainable
block4_conv2 is Non-Trainable
block4_conv3 is Non-Trainable
block4_pool is Non-Trainable
block5_conv1 is Non-Trainable
block5_conv2 is Non-Trainable
block5_conv3 is Trainable
block5_pool is Non-Trainable
global_average_pooling2d is Non-Trainable


In [82]:
model_vgg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0

In [0]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
model_vgg.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [84]:
model_vgg.fit(train_generator,
              steps_per_epoch=train_generator.n//128,
              validation_data=val_generator,
              validation_steps=val_generator.n//128,
              epochs=50,
              callbacks=[model_vgg_checkpoint])

Epoch 1/50
63/63 [==============================] - ETA: 0s - loss: 2.3519 - accuracy: 0.3855
Epoch 00001: val_accuracy did not improve from 0.33906
63/63 [==============================] - 31s 496ms/step - loss: 2.3519 - accuracy: 0.3855 - val_loss: 3.4008 - val_accuracy: 0.2589
Epoch 2/50
63/63 [==============================] - ETA: 0s - loss: 1.9179 - accuracy: 0.4710
Epoch 00002: val_accuracy did not improve from 0.33906
63/63 [==============================] - 31s 494ms/step - loss: 1.9179 - accuracy: 0.4710 - val_loss: 3.0508 - val_accuracy: 0.3161
Epoch 3/50
63/63 [==============================] - ETA: 0s - loss: 1.7082 - accuracy: 0.5223
Epoch 00003: val_accuracy improved from 0.33906 to 0.34062, saving model to dog_breed_classification_best.h5
63/63 [==============================] - 32s 508ms/step - loss: 1.7082 - accuracy: 0.5223 - val_loss: 2.7751 - val_accuracy: 0.3406
Epoch 4/50
63/63 [==============================] - ETA: 0s - loss: 1.5926 - accuracy: 0.5488
Epoch 000

## Summary
### After 50 epochs the Training accuracy has improved to __91%__ and Validation accuracy has improved to __40%__